In [3]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# SentenceBERT 모델 로드

In [4]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510455 -0.77338403  0.592771   ...  0.57923526  0.32683465
  -0.65089667]
 [-0.09361672 -0.18191531 -0.19230805 ... -0.03165794  0.30412534
  -0.26793587]]


# 데이터셋 로드

웰니스 대화 스크립트 데이터셋

https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-006

In [5]:
df = pd.read_csv('wellness_dataset_original.csv')

df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


# 전처리

In [6]:
df = df.drop(columns=['Unnamed: 3'])

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [7]:
df = df[~df['챗봇'].isna()]

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 송이에게도 상처를 주잖아요.


In [8]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [9]:
model.encode(df.loc[0, '유저'])

array([-4.80606616e-01, -2.94869214e-01,  4.37900215e-01, -6.40137792e-01,
        3.28669734e-02, -3.42647463e-01, -5.47484159e-02,  1.73055734e-02,
       -4.08220530e-01, -5.06034195e-01, -1.68733463e-01, -3.98676872e-01,
       -1.24776639e-01, -9.71538201e-02, -1.65286690e-01,  5.72622241e-03,
        6.13692179e-02, -1.91312209e-01,  2.53917515e-01, -5.85019588e-01,
       -2.84425616e-01, -2.32034832e-01, -3.27080876e-01,  6.72978386e-02,
       -1.63657332e-05, -4.72336441e-01, -3.60021919e-01,  2.91879863e-01,
       -6.63861394e-01, -3.10574591e-01,  5.79525113e-01, -3.11722845e-01,
        1.47699313e-02, -2.12172583e-01,  2.22058266e-01, -1.73828945e-01,
       -3.78458351e-01, -4.20398384e-01, -2.38218620e-01,  6.38703927e-02,
       -1.15304284e-01, -2.44564101e-01, -5.00228226e-01,  1.68355420e-01,
       -6.58360362e-01, -8.91942024e-01, -6.26956999e-01, -3.21965545e-01,
       -7.05358684e-01,  3.71447384e-01, -5.45803130e-01,  7.76298419e-02,
        1.09864593e-01,  

# 유저 대화내용 인코딩

In [10]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.2948692, 0.43790022, -0.64013..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561577, -0.14506267, 0.2949036, -0.673949..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652002, -0.08126839, 1.0945568, 0.1057922..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285072, -0.5076043..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 송이에게도 상처를 주잖아요.,"[-0.20277722, -0.37413925, 0.040532343, -0.862..."


In [11]:
df.to_csv('wellness_dataset.csv', index=False)

# 간단한 챗봇

In [12]:
text = '요즘 머리가 아프고 너무 힘들어'

embedding = model.encode(text)

In [13]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,구분,유저,챗봇,embedding,distance
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.48060662, -0.2948692, 0.43790022, -0.64013...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561577, -0.14506267, 0.2949036, -0.673949...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.6652002, -0.08126839, 1.0945568, 0.1057922...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.76790583, 0.4652073, 0.5285072, -0.5076043...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 송이에게도 상처를 주잖아요.,"[-0.20277722, -0.37413925, 0.040532343, -0.862...",0.315118


In [14]:
answer = df.loc[df['distance'].idxmax()]

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['distance'])

구분 증상/편두통
유사한 질문 요즘은 머리가 한쪽만 지그시 누르는 것처럼 무겁고 아파요.
챗봇 답변 으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.
유사도 0.8296287059783936
